# Data preparation for Object detection API

<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/ashishkrb7/Object-Detection-TF-data-preparation/blob/main/Data_Preparation_by_Ashish.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/ashishkrb7/Object-Detection-TF-data-preparation/blob/main/Data_Preparation_by_Ashish.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [ ]:
# %cd /content/
# import shutil
# import os
# shutil.rmtree("Object-Detection-TF-datapreparation")
# shutil.rmtree("Object-Detection-TF-datapreparation.gitdata")
# shutil.rmtree("Object-Detection-TF-datapreparationdata")
# shutil.rmtree("models")

In [ ]:
!python -m pip install tf_slim
!python -m pip install tf-models-official

In [ ]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
repo_url = 'https://github.com/ashishkrb7/Object-Detection-TF-data-preparation.git'
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url))).split(".")[0]
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

In [ ]:
%cd /content

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

In [ ]:
if not os.path.exists(repo_dir_path+'/data/images/train'):os.makedirs(repo_dir_path+'/data/images/train')
if not os.path.exists(repo_dir_path+'/data/images/test'):os.makedirs(repo_dir_path+'/data/images/test')
if not os.path.exists(repo_dir_path+'/data/annotations'):os.makedirs(repo_dir_path+'/data/annotations')

---

# Upload the dataset in train and test folder

---

In [ ]:
from google.colab import files

In [ ]:
%cd {repo_dir_path+'/data/images/train'}
upload=files.upload()

In [ ]:
!unzip train.zip
!rm train.zip

In [ ]:
%cd {repo_dir_path+'/data/images/test'}
upload=files.upload()

In [ ]:
!unzip test.zip
!rm test.zip

# Generate processed dataset

In [ ]:
%cd {repo_dir_path}

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

# Download dataset

In [ ]:
inputDir=repo_dir_path+'/data/annotations/'
%cd {inputDir}
!zip -r data.zip {inputDir}

In [ ]:
from google.colab import files
files.download('data.zip')